In [1]:
import gower
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from scipy.stats import chi2
from sklearn.manifold import TSNE
from pyod.models.kde import KDE
from scipy.spatial.distance import pdist as pdist
from scipy.spatial.distance import squareform as sf
from sklearn.neighbors import LocalOutlierFactor
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import MinMaxScaler
from kneed import KneeLocator
from sklearn.neighbors import NearestNeighbors as knn
from sklearn.metrics import silhouette_score, rand_score, adjusted_rand_score

In [2]:
labels = pd.read_csv("Data/BENCHMARK_outliers.csv")
trueLabels = pd.read_csv("Data/data2_preprocessed.csv")
trueLabels = trueLabels['anomalies']
labels = labels['SharpOutliers']

rand_score(trueLabels, labels)

0.8626325951705561